### Extração de dados youtube- visualizacão de videos

**Bibliotecas:**

* pandas
* numpy
* BeautifulSoup
* matplotlib


> Requer instalação da  **beautifulsoup4** para ler e extrair dados html, **tags**.

In [ ]:
 pip install beautifulsoup4

> Para gerar nuvem de palavras vou utilizar a biblioteca **stylecloud** para isso

**Imports de bliotecas para utilização:**

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import codecs
from datetime import datetime


***Local do arquivo html***

In [ ]:
local_html = '/home/mateus/Downloads/youtube/histórico/histórico-de-visualização.html'

**Instância de objeto da biblioteca BeautifulSoup**

In [ ]:
def cria_objeto_soup(html_doc):
    fp = codecs.open(html_doc, 'r') 
    sp = BeautifulSoup(fp)
    return sp

In [ ]:
def coleta_divs(nome_div,obj):
    mydivs = obj.findAll("div", {"class": nome_div})
    return  mydivs

In [ ]:
def cria_dicionario():
    informacoe_visualizacao = {
                                        'canal':[],
                                        'url_canal':[],
                                        'nome_video':[],
                                        'url_video':[],
                                        'data_visualizacao':[]
    }
    return informacoe_visualizacao

In [ ]:
nome_divs = 'content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1'
soup = cria_objeto_soup(local_html)
divs = coleta_divs(nome_divs,soup)


In [ ]:
dados = cria_dicionario()
for dado in divs:
    href_video = dado.a

    dados_novo_tag_a = dado.find_all('a')  
    date = str(dado).split('<br/>')
    tm_date = len(date)
    date = date[tm_date-1].split('</div>')
    txt = dado.text
    tm = len('Watched\xa0')
    txt = txt[tm:]
    
    
    try: 
       dados['canal'].append(dados_novo_tag_a[1].text)
    except:
       dados['canal'].append(np.NaN)
    
    try: 
       dados['url_canal'].append(dados_novo_tag_a[1]['href'])
    except:
       dados['url_canal'].append(np.NaN) 
    
    try:
       dados['nome_video'].append(txt) 
    except:
       dados['nome_video'].append(np.NaN)
    
    try:
       dados['url_video'].append(href_video['href'])
    except:
       dados['url_video'].append(np.NaN)
    
    try:  
       dados['data_visualizacao'].append(str(date[0]))
    except:
       dados['data_visualizacao'].append(np.NaN)
  

In [ ]:
visualizao_youtube = pd.DataFrame(dados)

In [ ]:
visualizao_youtube.head(10)

In [ ]:
visualizao_youtube.info()


In [ ]:
visualizao_youtube.head()


In [ ]:
meses = {
        '01':'jan.',
        '02':'fev.',
        '03':'mar.',
        '04':'abr.',
        '05':'mai.',
        '06':'jun.',
        '07':'jul.',
        '08':'ago.',
        '09':'set.',
        '10':'out.',
        '11':'nov.',
        '12':'dez.',
}


In [ ]:
  def limpa_date(string):
    
    string = list(string)
    
    string =  ''.join(string).strip()
    string = string.split(' ')
   
    for i,valor in enumerate(string):
        if valor =='de' or valor == 'BRT':
            string.pop(i)     
        if i == 2:
            
            for chave in meses.keys():
                if meses[chave] == string[1].lower():
                    string[1] = chave
    string.insert(1,'/')
    string.insert(3,'/')
    string = ' '.join(str(v) for v in string) 
    return string



In [ ]:
#Aplica funcao para aumentar letra do mes
visualizao_youtube['data_visualizacao'] = visualizao_youtube['data_visualizacao'].apply(limpa_date)

In [ ]:
visualizao_youtube.head()

In [ ]:
visualizao_youtube['data_visualizacao'] = pd.to_datetime(visualizao_youtube['data_visualizacao'],format  = '%d / %m / %Y %H:%M:%S')

In [ ]:
visualizao_youtube.head()

In [ ]:
#Excluir valores nulos de registro
#de visualização, pois eles são na maioria videos ou 
#canais que foi removido ou é privado,não conta para mim, pois não mostra nome video e nome do canal.
youtube_dado_limpo = visualizao_youtube.dropna()

In [ ]:
youtube_dado_limpo.head()

In [ ]:
#salavando dados como csv para usar no tableau
youtube_dado_limpo.to_csv('/home/mateus/Downloads/youtube/visualizacao.csv',index = False)


#### ***Dados de Busca no you tube***

##### Objetivo
* Coletar **informações** do registro de busca do arquivo do you tube



In [ ]:
local_html = '/home/mateus/Downloads/youtube/histórico/histórico de pesquisa.html'

In [ ]:
#Dicionario para guardar dados do arquivo html referente a data e busca
dict_seacher = {
               'searhed_for':[],
                'date':[]
}

In [ ]:
#Escopo para criar objeto soup , ou sejam objeto criado pela biblioteca BeautifulSoup para trabalhar com  html
nome_div = "content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1"#Nome da div que quero busca para pegar meus dados
soup = cria_objeto_soup(local_html) #Cria objeto
divs = coleta_divs(nome_div,soup)#Função para buscar dado que compõe a div

In [ ]:
#Extração e add de dados no dicionario conforme a regra de negocio implentada
for dado in divs:

   
    texto = dado.text
    tm_texto = len(texto)
    sub = tm_texto-31
    texto_date_busca = texto[13:tm_texto]
    tm_texto_2 = len(texto_date_busca)
   
 
    
    try: 
       dict_seacher['searhed_for'].append(texto_date_busca[0:sub] )
    except:
        dict_seacher['searhed_for'].append(np.NaN)
    
    try: 
       dict_seacher['date'].append(texto[sub:tm_texto:])
    except:
       dict_seacher['date'].append(np.NaN) 
    
    

In [ ]:
#Geração do dataframe com base no dicionario criado anteriomente
searche = pd.DataFrame(dict_seacher)
searche.head(10)

In [ ]:
#Função para retirar dados sujos provido da extração da coluna searcher
def retira_data_searche(x):
    dig =[str(i) for i in range(10)]
    x = list(x)
    busca = []
    for i,letra in enumerate(x):
        if letra in dig      :
           break
        else:
           busca.append(letra) 
    return ''.join(busca)
    
     

In [ ]:
#Função para retirar numeros que vem antes da data e hora do registro
def remove_primeiro_digito(x):
    lista = list(x)
 
    for i,letra  in enumerate(lista):
        if lista[0].isalpha():
            lista.pop(0)
          
        else:
            if (str(lista[0]) in ['3','4','5','6','7','8','9']):
                    if (lista[1] =='/') or (lista[1] == ' '):
                           return ''.join(lista)
                    else:
                           try:
                                num1 = int(lista[0])
                                num2 = int(lista[1])
                                if (sum([num1,num2]) > 4):
                                    lista.pop(0)
                           except:     
                                return ''.join(lista)   
        return ''.join(lista)                           
                   
     

In [ ]:
searche.info()

In [ ]:
#Como a forma que trouxe o dado da extração sempre iria vim com uma letra junta com data ou virsr versa,
#Percebi que se eu usar uma função para verificar se o primeiro digito e uma letra, removo essa letra ele
searche['date'].tail(20)

In [ ]:
#aplicando a função a cada registro para limpeza da datatime que vem com string na sua primeiras posições
searche['date'] = searche['date'].apply(remove_primeiro_digito)

In [ ]:
searche.tail()

In [ ]:
#aplicando a função a cada registro para limpeza da datetime para retirar string e aplicar numero para mes

searche['date'] = searche['date'].apply(limpa_date)


In [ ]:
#Fazendo conversão para datetime pandas commetod to_datetime
searche['date'] = pd.to_datetime(searche['date'],format  = '%d / %m / %Y %H:%M:%S')

In [ ]:
#Visualizando
searche.head()

In [ ]:
searche.info()

In [ ]:
#Vamos limpar a consulta e deixar apenas palavras chaves
searche['searhed_for'] = searche['searhed_for'].apply(retira_data_searche)


In [ ]:
searche.info()

In [ ]:
searche.head()

In [ ]:
#salavando dados como csv para usar no tableau
searche.to_csv('/home/mateus/Downloads/youtube/searche.csv',index = False)

In [ ]:
stylecloud.gen_stylecloud(file_path='constitution.txt',
                          icon_name='fas fa-dog',
                          palette='colorbrewer.diverging.Spectral_11',
                          background_color='green',
                          gradient='horizontal')